<a href="https://colab.research.google.com/github/NastasiaMazur/VU_1_1/blob/main/exercises/HomeExercise3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Home Exercise 3: Creating Neural Model in PyTorch

In this home exercise, you will first learn how to create a neural model in PyTorch and then you will train and improve a mini-implementation of an embedding model.


---

## **Exercise 3a: Neural Network Model**

First we need to import the neural network module of PyTorch:

In [84]:
import torch
import torch.nn as nn

We can use `nn.Linear(H_in, H_out)` to create a a linear layer. This will take a matrix of `(N, *, H_in)` dimensions and output a matrix of `(N, *, H_out)`. The `*` denotes that there could be arbitrary number of dimensions in between. The linear layer performs the operation `Ax+b`, where `A` and `b` are initialized randomly. If we don't want the linear layer to learn the bias parameters, we can initialize our layer with `bias=False`.

In [85]:
# Create the inputs
input = torch.ones(2,3,4)
print("Input ", input)

# N* H_in -> N*H_out
linear = nn.Linear(4, 2)
linear_output = linear(input)
linear_output

Input  tensor([[[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]])


tensor([[[-0.0176,  0.3654],
         [-0.0176,  0.3654],
         [-0.0176,  0.3654]],

        [[-0.0176,  0.3654],
         [-0.0176,  0.3654],
         [-0.0176,  0.3654]]], grad_fn=<ViewBackward0>)

In [86]:
list(linear.parameters()) # Ax + b

[Parameter containing:
 tensor([[-0.4753,  0.4576, -0.1313,  0.4969],
         [ 0.0978,  0.3345, -0.0681,  0.1483]], requires_grad=True),
 Parameter containing:
 tensor([-0.3656, -0.1471], requires_grad=True)]

Let's add an activation function:

In [87]:
sigmoid = nn.Sigmoid()
output = sigmoid(linear_output)
output

tensor([[[0.4956, 0.5903],
         [0.4956, 0.5903],
         [0.4956, 0.5903]],

        [[0.4956, 0.5903],
         [0.4956, 0.5903],
         [0.4956, 0.5903]]], grad_fn=<SigmoidBackward0>)

Instead of creating intermediate layers and passing variables around, we can create a sequence:

In [88]:
block = nn.Sequential(
    nn.Linear(4, 2),
    nn.Sigmoid()
)

input = torch.ones(2,3,4)
output = block(input)
output

tensor([[[0.2633, 0.8530],
         [0.2633, 0.8530],
         [0.2633, 0.8530]],

        [[0.2633, 0.8530],
         [0.2633, 0.8530],
         [0.2633, 0.8530]]], grad_fn=<SigmoidBackward0>)

---

## **Exercise 3b: Word Embeddings**


Instead of using predefined modules of nn we can define our own modules and build custom neural networks. As a toy example, we will convert words to word embeddings. The preprocessing below should be done more elegantly and not as simply as below.

In [89]:
import string

# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold."""

training_sentence = test_sentence.translate(str.maketrans('', '', string.punctuation)).lower().split()
print(training_sentence)

['when', 'forty', 'winters', 'shall', 'besiege', 'thy', 'brow', 'and', 'dig', 'deep', 'trenches', 'in', 'thy', 'beautys', 'field', 'thy', 'youths', 'proud', 'livery', 'so', 'gazed', 'on', 'now', 'will', 'be', 'a', 'totterd', 'weed', 'of', 'small', 'worth', 'held', 'then', 'being', 'asked', 'where', 'all', 'thy', 'beauty', 'lies', 'where', 'all', 'the', 'treasure', 'of', 'thy', 'lusty', 'days', 'to', 'say', 'within', 'thine', 'own', 'deep', 'sunken', 'eyes', 'were', 'an', 'alleating', 'shame', 'and', 'thriftless', 'praise', 'how', 'much', 'more', 'praise', 'deservd', 'thy', 'beautys', 'use', 'if', 'thou', 'couldst', 'answer', 'this', 'fair', 'child', 'of', 'mine', 'shall', 'sum', 'my', 'count', 'and', 'make', 'my', 'old', 'excuse', 'proving', 'his', 'beauty', 'by', 'succession', 'thine', 'this', 'were', 'to', 'be', 'new', 'made', 'when', 'thou', 'art', 'old', 'and', 'see', 'thy', 'blood', 'warm', 'when', 'thou', 'feelst', 'it', 'cold']


Next let's find our vocabulary, i.e., all the unique words in the training data:

In [90]:
vocabulary = set(w for w in training_sentence)
vocabulary

{'a',
 'all',
 'alleating',
 'an',
 'and',
 'answer',
 'art',
 'asked',
 'be',
 'beauty',
 'beautys',
 'being',
 'besiege',
 'blood',
 'brow',
 'by',
 'child',
 'cold',
 'couldst',
 'count',
 'days',
 'deep',
 'deservd',
 'dig',
 'excuse',
 'eyes',
 'fair',
 'feelst',
 'field',
 'forty',
 'gazed',
 'held',
 'his',
 'how',
 'if',
 'in',
 'it',
 'lies',
 'livery',
 'lusty',
 'made',
 'make',
 'mine',
 'more',
 'much',
 'my',
 'new',
 'now',
 'of',
 'old',
 'on',
 'own',
 'praise',
 'proud',
 'proving',
 'say',
 'see',
 'shall',
 'shame',
 'small',
 'so',
 'succession',
 'sum',
 'sunken',
 'the',
 'then',
 'thine',
 'this',
 'thou',
 'thriftless',
 'thy',
 'to',
 'totterd',
 'treasure',
 'trenches',
 'use',
 'warm',
 'weed',
 'were',
 'when',
 'where',
 'will',
 'winters',
 'within',
 'worth',
 'youths'}

We introduce a special token, `<unk>`, to tackle the words that are out of vocabulary. We could pick another string for our unknown token if we wanted. The only requirement here is that our token should be unique: we should only be using this token for unknown words. We will also add this special token to our vocabulary.

In [91]:
vocabulary.add("<unk>")

Now we will create the index for our vocabulary - one index to word and one word to index to make looking up words easier:

In [92]:
ix_to_word = sorted(list(vocabulary))
word_to_ix = {word: ind for ind, word in enumerate(ix_to_word)}
word_to_ix

{'<unk>': 0,
 'a': 1,
 'all': 2,
 'alleating': 3,
 'an': 4,
 'and': 5,
 'answer': 6,
 'art': 7,
 'asked': 8,
 'be': 9,
 'beauty': 10,
 'beautys': 11,
 'being': 12,
 'besiege': 13,
 'blood': 14,
 'brow': 15,
 'by': 16,
 'child': 17,
 'cold': 18,
 'couldst': 19,
 'count': 20,
 'days': 21,
 'deep': 22,
 'deservd': 23,
 'dig': 24,
 'excuse': 25,
 'eyes': 26,
 'fair': 27,
 'feelst': 28,
 'field': 29,
 'forty': 30,
 'gazed': 31,
 'held': 32,
 'his': 33,
 'how': 34,
 'if': 35,
 'in': 36,
 'it': 37,
 'lies': 38,
 'livery': 39,
 'lusty': 40,
 'made': 41,
 'make': 42,
 'mine': 43,
 'more': 44,
 'much': 45,
 'my': 46,
 'new': 47,
 'now': 48,
 'of': 49,
 'old': 50,
 'on': 51,
 'own': 52,
 'praise': 53,
 'proud': 54,
 'proving': 55,
 'say': 56,
 'see': 57,
 'shall': 58,
 'shame': 59,
 'small': 60,
 'so': 61,
 'succession': 62,
 'sum': 63,
 'sunken': 64,
 'the': 65,
 'then': 66,
 'thine': 67,
 'this': 68,
 'thou': 69,
 'thriftless': 70,
 'thy': 71,
 'to': 72,
 'totterd': 73,
 'treasure': 74,
 'trenc

👋 ⚒ How can we now lookup which word is the fifth word in our index list?

In [93]:
# Your code here
fifth_ix = 4
fifth_word = [word for word, ix in word_to_ix.items() if ix == fifth_ix][0]
fifth_word
# I considered the 0 index as an OOV word


'an'

We will use a very simple solution of building trigrams to train our model.

In [94]:
trigrams = [([training_sentence[i], training_sentence[i + 1]], training_sentence[i + 2])
            for i in range(len(training_sentence)-2)]
print(trigrams)

[(['when', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege'), (['shall', 'besiege'], 'thy'), (['besiege', 'thy'], 'brow'), (['thy', 'brow'], 'and'), (['brow', 'and'], 'dig'), (['and', 'dig'], 'deep'), (['dig', 'deep'], 'trenches'), (['deep', 'trenches'], 'in'), (['trenches', 'in'], 'thy'), (['in', 'thy'], 'beautys'), (['thy', 'beautys'], 'field'), (['beautys', 'field'], 'thy'), (['field', 'thy'], 'youths'), (['thy', 'youths'], 'proud'), (['youths', 'proud'], 'livery'), (['proud', 'livery'], 'so'), (['livery', 'so'], 'gazed'), (['so', 'gazed'], 'on'), (['gazed', 'on'], 'now'), (['on', 'now'], 'will'), (['now', 'will'], 'be'), (['will', 'be'], 'a'), (['be', 'a'], 'totterd'), (['a', 'totterd'], 'weed'), (['totterd', 'weed'], 'of'), (['weed', 'of'], 'small'), (['of', 'small'], 'worth'), (['small', 'worth'], 'held'), (['worth', 'held'], 'then'), (['held', 'then'], 'being'), (['then', 'being'], 'asked'), (['being', 'asked'], 'where'), (['asked', 'where'

In [95]:
class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, window_size, hidden_dim):
        super(NGramLanguageModeler, self).__init__()          # calls the constructor of the parent class (nn.Module) to initialize the model

        self.embeddings = nn.Embedding(vocab_size, embedding_dim) # creates an embedding layer;  converts input indices into dense vectors
        self.hidden_layer = nn.Sequential(                        # hidden layer of the model as a sequential container
            nn.Linear(window_size * embedding_dim, hidden_dim),
            nn.ReLU()
        )
        self.output_layer = nn.Linear(hidden_dim, vocab_size)     # creates the output layer; takes the hidden representation and produces output scores for each word in the vocabulary

    def forward(self, inputs):                            # specifies how the input data should be processed through the layers to produce the output
        embeds = self.embeddings(inputs).view((1, -1))    # takes the input indices inputs, passes them through the embedding layer, and flattens the resulting tensor into a 1D tensor
        layer1 = self.hidden_layer(embeds)                # passes the flattened embeddings through the hidden layer, applying the linear transformation followed by the ReLU activation
        output = self.output_layer(layer1)                # passes the result of the hidden layer through the output layer, producing the final output scores for each word in the vocabulary
        probabilities = nn.functional.log_softmax(output, dim=1)
        return probabilities

👋 ⚒ Now let's train the model. Try to adapt the hyperparamters so that the total_loss is reduced. These include in the initial settings:

*   Dimensionality of the embeddings: 10
*   Dimensionality of the hidden layer: 128
*   Number of epochs: 10
*   Learning rate: 0.002
*   Loss function (`NLLLoss()` Negative Log Likelihood right now)

The window size can in this toy example not be changed, since we train on trigrams (so we only have two context words in the set).

The code cell before, the `class NGramLanguageModeler(nn.Module)` defines the exact setup of the network. For the setup one thing to change could be the  activation functions (ReLU and Log-Softmax right now) - others can be found [here](https://pytorch.org/docs/stable/nn.html) for sequential layers and here for other uses as [nn.functional](https://pytorch.org/docs/stable/nn.functional.html)).

Just for comparison, this is the initial output with the first settings of the notebook:

`See how loss decreases with each epoch:  [4.502952638980561, 4.453238930322428, 4.404419487556525, 4.356399970771992, 4.309086953644204, 4.262444785210938, 4.216389432417608, 4.170906921403598, 4.125928629816106, 4.081349758975274]
Loss of the last epoch: 4.081349758975274`




In [97]:
window_size = 2
#embedding_dim = 10
embedding_dim = 50
#hidden_dim = 128
hidden_dim = 256
num_epochs = 10

losses = []
# Negative log likelihood
loss_function = nn.NLLLoss()
ngram_model = NGramLanguageModeler(len(vocabulary), embedding_dim, window_size, hidden_dim)

# What do SGD and lr mean? What happenes if you change them?
optimizer = torch.optim.SGD(ngram_model.parameters(), lr=0.002)

for epoch in range(num_epochs):
    total_loss = 0
    for context, target in trigrams:
        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        optimizer.zero_grad()

        # Step 3. Run the forward pass, getting probabilities over next
        # words - the size of this tensor is 87 corresponding to the size of the vocabulary
        probabilities = ngram_model.forward(context_idxs)

        # Step 4. Compute your loss function. The target word (gold standard label) needs to be
        # wrapped in a tensor.
        loss = loss_function(probabilities, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss / len(trigrams))

print("See how loss decreases with each epoch: ", losses)
print("Loss of the last epoch:", losses[-1])

See how loss decreases with each epoch:  [4.49342289646115, 4.388323336575938, 4.285124076151215, 4.183570576980051, 4.083265519775121, 3.983727045818768, 3.884790912138677, 3.786367958625861, 3.688175365988132, 3.590011345601715]
Loss of the last epoch: 3.590011345601715


👋 ⚒ Implement the cosine simmilarity for the trained embeddings. For this you need to choose too find the index of the two words you wish to compare, get their embeddings, convert them into numpy arrays, and run them through the cosine function provided.

In [104]:
import numpy as np # my code

# Tensor to look up specific embeddings
lookup_tensor = torch.tensor(list(word_to_ix.values()), dtype=torch.long)
print(lookup_tensor) # my code

# Embedding for the first word in the vocabulary
lookup_embeds = ngram_model.embeddings(lookup_tensor)[0]
print(lookup_embeds)
#cosine = np.dot(A,B)/(norm(A, axis=1)*norm(B))

# Example: Calculate cosine similarity between the embeddings of two words # my code
word1_index = word_to_ix["thy"] # my code
word2_index = word_to_ix["beauty"] # my code
# Get embeddings # my code
embedding_word1 = ngram_model.embeddings(torch.tensor([word1_index], dtype=torch.long))[0].detach().numpy() # my code
embedding_word2 = ngram_model.embeddings(torch.tensor([word2_index], dtype=torch.long))[0].detach().numpy() # my code

# my code:
def cosine_similarity(A, B):
    dot_product = np.dot(A, B)
    norm_A = np.linalg.norm(A)
    norm_B = np.linalg.norm(B)
    similarity = dot_product / (norm_A * norm_B)
    return similarity

# Calculate cosine similarity # my code
cosine_similarity_value = cosine_similarity(embedding_word1, embedding_word2) # my code
print("Cosine Similarity between 2 words:", cosine_similarity_value) # my code



tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
        54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71,
        72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86])
tensor([ 2.8897, -1.6840,  0.4014,  0.1840,  0.0541, -0.7877, -0.0118,  0.9142,
        -0.2238,  0.8519,  0.8934,  3.1170, -0.0438,  1.9054,  0.2303, -1.4030,
        -2.0621,  0.0404, -0.0242,  0.5947, -0.6109, -0.8753, -0.4947, -0.1240,
        -1.2156, -1.3851,  0.0031,  0.7782, -0.9306,  0.4787, -0.9150, -0.4600,
        -1.0714, -0.0929,  1.9805,  1.0161,  1.5828, -0.7693,  0.4117, -0.7375,
        -1.5565, -0.0306, -0.2960, -0.3483, -0.1584, -0.0138, -0.2453,  0.6473,
         0.0519,  0.3841], grad_fn=<SelectBackward0>)
Cosine Similarity between 2 words: -0.07576379


In [109]:
import torch

word1 = "beauty"
word2 = "treasure"

index_word1 = word_to_ix[word1]
index_word2 = word_to_ix[word2]

# Tensor to look up specific embeddings
lookup_tensor = torch.tensor(list(word_to_ix.values()), dtype=torch.long)

# Embedding for the first word in the vocabulary
lookup_embeds = ngram_model.embeddings(lookup_tensor)[0]
print(lookup_embeds)

#Convert embeddings into numpy arrays: #my code
embed_word1 = lookup_embeds[0].detach().numpy()
embed_word2 = lookup_embeds[1].detach().numpy()

# Calculate dot product #my code
dot_product = np.dot(embed_word1, embed_word2)
# Calculate norms #my code
norm_word1 = np.sqrt(np.sum(embed_word1**2))
norm_word2 = np.sqrt(np.sum(embed_word2**2))

# Calculate cosine similarity #my code
cosine_similarity = dot_product / (norm_word1 * norm_word2)
print(cosine_similarity)


#cosine_similarity = np.dot(embed_word1, embed_word2) / (np.linalg.norm(embed_word1) * np.linalg.norm(embed_word2))
#print(f"Cosine Similarity between '{word1}' and '{word2}': {cosine_similarity}")

#cosine = np.dot(A,B)/(norm(A, axis=1)*norm(B))

tensor([ 2.8897, -1.6840,  0.4014,  0.1840,  0.0541, -0.7877, -0.0118,  0.9142,
        -0.2238,  0.8519,  0.8934,  3.1170, -0.0438,  1.9054,  0.2303, -1.4030,
        -2.0621,  0.0404, -0.0242,  0.5947, -0.6109, -0.8753, -0.4947, -0.1240,
        -1.2156, -1.3851,  0.0031,  0.7782, -0.9306,  0.4787, -0.9150, -0.4600,
        -1.0714, -0.0929,  1.9805,  1.0161,  1.5828, -0.7693,  0.4117, -0.7375,
        -1.5565, -0.0306, -0.2960, -0.3483, -0.1584, -0.0138, -0.2453,  0.6473,
         0.0519,  0.3841], grad_fn=<SelectBackward0>)
-1.0


In [ ]:
# Tensor to look up specific embeddings
lookup_tensor = torch.tensor(list(word_to_ix.values()), dtype=torch.long)

# Embedding for the first word in the vocabulary
lookup_embeds = ngram_model.embeddings(lookup_tensor)[0]
print(lookup_embeds)
#cosine = np.dot(A,B)/(norm(A, axis=1)*norm(B))